In [1]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from vit_meta import vit_small
from torchvision.transforms import v2
from PIL import Image
import torch
import numpy as np




c:\Users\lucat\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\onnx\_internal\_beartype.py:35: UserWarning: unhashable type: 'list'
  warnings.warn(f"{e}")


In [2]:
DEFAULT_MEAN = (0.485, 0.456, 0.406)
DEFAULT_STD = (0.229, 0.224, 0.225)
transform = v2.Compose([
    v2.ToImage(), 
    v2.Resize((224, 224)),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=DEFAULT_MEAN, std=DEFAULT_STD)])

In [3]:
image_path= 'C:/Users/lucat/Documents/MEDMINST/data_split/breastmnist_224/test/1/test2_1.png'
image = Image.open(image_path)
rgb_img = image.convert('RGB')
#The input image should np.float32 in the range [0, 1]
rgb_img =  np.array(rgb_img) / 255.0
tensor = transform(image).unsqueeze(0)
print(tensor.shape)
print(rgb_img.size)


torch.Size([1, 3, 224, 224])
150528


In [4]:
model= vit_small()
wp = 'F:/sdeex/sdeex/configs/dino_radio/trained_models3/breastmnist/dino_vits16/best.pth'
state_dict = torch.load(wp, map_location="cpu")
model.load_state_dict(state_dict, strict=False)
print(model.blocks[-1].norm1)
target_layers = [model.blocks[-1].norm1]


LayerNorm((384,), eps=1e-06, elementwise_affine=True)


In [5]:

# We have to specify the target we want to generate the CAM for.
targets = None
print(targets)
# Construct the CAM object once, and then re-use it on many images.
with GradCAM(model=model, target_layers=target_layers) as cam:
  # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
  print(tensor.shape)
  grayscale_cam = cam(input_tensor=tensor, targets=targets)
  print(grayscale_cam.shape)
  # In this example grayscale_cam has only one image in the batch:
  grayscale_cam = grayscale_cam[0, :]
  print(grayscale_cam.shape)
  visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
  
  # You can also get the model outputs without having to redo inference


None
torch.Size([1, 3, 224, 224])
(1, 197, 384)
An exception occurred in CAM with block: <class 'numpy.exceptions.AxisError'>. Message: axis 3 is out of bounds for array of dimension 3


In [6]:
#plotting
import matplotlib.pyplot as plt
plt.imshow(visualization)
plt.show()


NameError: name 'visualization' is not defined